<hr>
# PYTHON FOR DATA SCIENCE
### Exploring a Real World Example

#Agenda

- <p style="color: red">Define the problem and the approach</p>
- Data basics: loading data, looking at your data, basic commands
- Handling missing values
- Intro to scikit-learn
- Grouping and aggregating data
- Feature selection
- Fitting and evaluating a model
- Deploying your work


##In this notebook you will

- Determine if the problem is worth solving
- Define an approach
- Develop a workflow outline

<hr>
## Revisiting Kaggle's Give Me Some Credit Competition

### Intro

#### We'll be exploring an age-old prediction problem--predicting risk on consumer loans.

#### We've selected this topic as a case study because the problem is well-defined and familiar to most people. Additionally, this is a binary classification problem, so the strategies we apply should be relatively extensible to other problems you may encounter totally unrelated to credit and finance.

### About the data

The data is made available to us by Kaggle and was used in a competition in 2011.

[http://www.kaggle.com/c/GiveMeSomeCredit](http://www.kaggle.com/c/GiveMeSomeCredit)

### About the prediction problem

Predict the probability that somebody will experience financial distress in the next two years.

## Developing an understanding of the data and the problem

#### Key questions we'll need to keep in mind

- How do losses occur?
- What are the characteristics that constitute credit default?
- How often do people "go bad?"
- How might we improve loss rates?

In [149]:
import pandas as pd
df = pd.read_csv("./data/credit-training.csv")


# 
# Column Types?
print df.dtypes

# Missing Value?
# find columns that have null values
print df.isnull().any(axis=0)
# looks like MonthlyIncome has a lot of missing value
print df.shape
print "# null values in MonthlyIncome: %i" % df['MonthlyIncome'].isnull().sum()
print "# null values in NumberOfDependents: %i" % sum(df['NumberOfDependents'].isnull() == True)
df[df.isnull().any(axis=1)]

# Summary?
df.describe()
df.info() # info() is a faster way to spot null values


SeriousDlqin2yrs                          int64
RevolvingUtilizationOfUnsecuredLines    float64
age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
MonthlyIncome                           float64
NumberOfOpenCreditLinesAndLoans           int64
NumberOfTimes90DaysLate                   int64
NumberRealEstateLoansOrLines              int64
NumberOfTime60-89DaysPastDueNotWorse      int64
NumberOfDependents                      float64
dtype: object
SeriousDlqin2yrs                        False
RevolvingUtilizationOfUnsecuredLines    False
age                                     False
NumberOfTime30-59DaysPastDueNotWorse    False
DebtRatio                               False
MonthlyIncome                            True
NumberOfOpenCreditLinesAndLoans         False
NumberOfTimes90DaysLate                 False
NumberRealEstateLoansOrLines            False
NumberOfTime60-89DaysPastDueNotWorse    Fals

In [150]:
# inspect unique and unique counts
print df['NumberOfDependents'].nunique()
print df['NumberOfDependents'].unique()

for col in df.columns:
    print "column %s" % col
    print df[col].value_counts(sort=True).head(5)

13
[  2.   1.   0.  nan   3.   4.   5.   6.   8.   7.  20.  10.   9.  13.]
column SeriousDlqin2yrs
0    139974
1     10026
dtype: int64
column RevolvingUtilizationOfUnsecuredLines
0.000000    10878
1.000000    10256
1.000000       17
0.950100        8
0.954092        6
dtype: int64
column age
49    3837
48    3806
50    3753
63    3719
47    3719
dtype: int64
column NumberOfTime30-59DaysPastDueNotWorse
0    126018
1     16033
2      4598
3      1754
4       747
dtype: int64
column DebtRatio
0    4113
1     229
4     174
2     170
3     162
dtype: int64
column MonthlyIncome
5000    2757
4000    2106
6000    1934
3000    1758
0       1634
dtype: int64
column NumberOfOpenCreditLinesAndLoans
6    13614
7    13245
5    12931
8    12562
4    11609
dtype: int64
column NumberOfTimes90DaysLate
0    141662
1      5243
2      1555
3       667
4       291
dtype: int64
column NumberRealEstateLoansOrLines
0    56188
1    52338
2    31522
3     6300
4     2170
dtype: int64
column NumberOfTime60-89Day

In [151]:
# crosstab is powerful
print df['SeriousDlqin2yrs'].value_counts()
print pd.crosstab(df['NumberOfDependents'],df['SeriousDlqin2yrs'])

0    139974
1     10026
dtype: int64
SeriousDlqin2yrs        0     1
NumberOfDependents             
0                   81807  5095
1                   24381  1935
2                   17938  1584
3                    8646   837
4                    2565   297
5                     678    68
6                     134    24
7                      46     5
8                      22     2
9                       5     0
10                      5     0
13                      1     0
20                      1     0


In [152]:
# Basic cleaning
import re
def camel_to_snake(column_name):
    """
    converts a string that is camelCase into snake_case
    Example:
        print camel_to_snake("javaLovesCamelCase")
        > java_loves_camel_case
    See Also:
        http://stackoverflow.com/questions/1175208/elegant-python-function-to-convert-camelcase-to-camel-case
    """
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', column_name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

df.columns = [camel_to_snake(col) for col in df.columns]
df.columns.tolist()

['serious_dlqin2yrs',
 'revolving_utilization_of_unsecured_lines',
 'age',
 'number_of_time30-59_days_past_due_not_worse',
 'debt_ratio',
 'monthly_income',
 'number_of_open_credit_lines_and_loans',
 'number_of_times90_days_late',
 'number_real_estate_loans_or_lines',
 'number_of_time60-89_days_past_due_not_worse',
 'number_of_dependents']

In [66]:
# Basic filtering
#mask = (df.age >= 35) & (df.serious_dlqin2yrs==0) & (df.number_of_open_credit_lines_and_loans < 10)


In [153]:
# Now we are official onto handling missing value...
#?pd.melt
# another way to count missing data
df_lng = pd.melt(df)
null_vals = df_lng.value.isnull()
print null_vals.sum() 
pd.crosstab(df_lng.variable, null_vals)


33655


value,False,True
variable,,
age,150000,0
debt_ratio,150000,0
monthly_income,120269,29731
number_of_dependents,146076,3924
number_of_open_credit_lines_and_loans,150000,0
number_of_time30-59_days_past_due_not_worse,150000,0
number_of_time60-89_days_past_due_not_worse,150000,0
number_of_times90_days_late,150000,0
number_real_estate_loans_or_lines,150000,0


In [154]:
# a toy example of melt
melted = pd.melt(df, id_vars=["serious_dlqin2yrs"], value_vars=["age", "debt_ratio"])
print len(melted)

300000


In [94]:
# pandas provides different ways to fill missing value
s = pd.Series([1, 2, None, 4])
s.fillna(3)
s.bfill()
s.ffill()
s.fillna(s.mean())

0    1.000000
1    2.000000
2    2.333333
3    4.000000
dtype: float64

In [155]:
# handle number_of_dependents
df.number_of_dependents.value_counts()
# we naively fill the missing value with the mode 
df.number_of_dependents = df.number_of_dependents.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Data columns (total 11 columns):
serious_dlqin2yrs                              150000 non-null int64
revolving_utilization_of_unsecured_lines       150000 non-null float64
age                                            150000 non-null int64
number_of_time30-59_days_past_due_not_worse    150000 non-null int64
debt_ratio                                     150000 non-null float64
monthly_income                                 120269 non-null float64
number_of_open_credit_lines_and_loans          150000 non-null int64
number_of_times90_days_late                    150000 non-null int64
number_real_estate_loans_or_lines              150000 non-null int64
number_of_time60-89_days_past_due_not_worse    150000 non-null int64
number_of_dependents                           150000 non-null float64
dtypes: float64(4), int64(7)
memory usage: 13.7 MB


In [164]:
# we need to take care of monthly_income more sophisticatedly..

# the naive method is to simply fill with median or mean
df.monthly_income.describe()

import numpy as np
# use imputation: we try to infer the value from other columns
# split data into train and test
is_test = np.random.uniform(0, 1, len(df)) > 0.75
train = df[is_test==False]
test = df[is_test==True]
print len(train), len(test)
# remember to apply the imputer to the test data

112524 37476


In [201]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

rows_train = train.monthly_income.notnull()
rows_test = test.monthly_income.notnull()

# get the correlation pairs
train[rows_train].corr().ix[:, 5]


# find the most correlated variables
cols_x = ['number_of_open_credit_lines_and_loans', 'number_of_dependents', 'number_real_estate_loans_or_lines']

train_not_null = train[cols_x][rows_train]
test_not_null = test[cols_x][rows_test]


lr = LinearRegression()
lr.fit(train_not_null, train['monthly_income'][rows_train])
print lr.score(test_not_null, test['monthly_income'][rows_test])
# score: 0.0478125755667
knn = KNeighborsRegressor(n_neighbors=120)
knn.fit(train_not_null, train['monthly_income'][rows_train])
print knn.score(test_not_null, test['monthly_income'][rows_test])
# score: 0.00680687486842

# use linear regression model as imputer

0.0478125755667
0.00680687486842


In [189]:
train[rows_train].corr().ix[:, 5]

serious_dlqin2yrs                             -0.018743
revolving_utilization_of_unsecured_lines       0.002847
age                                            0.034589
number_of_time30-59_days_past_due_not_worse   -0.009318
debt_ratio                                    -0.024013
monthly_income                                 1.000000
number_of_open_credit_lines_and_loans          0.084120
number_of_times90_days_late                   -0.011705
number_real_estate_loans_or_lines              0.115956
number_of_time60-89_days_past_due_not_worse   -0.010197
number_of_dependents                           0.057711
Name: monthly_income, dtype: float64

In [203]:
train_null = train[cols_x][~rows_train]
test_null = test[cols_x][~rows_test]

new_values_train = lr.predict(train_null)
train['monthly_income'][~rows_train] = new_values_train

new_values_test = lr.predict(test_null)
test['monthly_income'][~rows_test] = new_values_test

# ways to deal with the (false positive) warning
#train.loc[~rows_train, 'monthly_income']


C:\Users\Hong Xu\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Hong Xu\AppData\Local\Continuum\Anaconda\lib\site-packages\IPython\kernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'pandas.core.frame.DataFrame'>
Int64Index: 112524 entries, 0 to 149999
Data columns (total 11 columns):
serious_dlqin2yrs                              112524 non-null int64
revolving_utilization_of_unsecured_lines       112524 non-null float64
age                                            112524 non-null int64
number_of_time30-59_days_past_due_not_worse    112524 non-null int64
debt_ratio                                     112524 non-null float64
monthly_income                                 112524 non-null float64
number_of_open_credit_lines_and_loans          112524 non-null int64
number_of_times90_days_late                    112524 non-null int64
number_real_estate_loans_or_lines              112524 non-null int64
number_of_time60-89_days_past_due_not_worse    112524 non-null int64
number_of_dependents                           112524 non-null float64
dtypes: float64(4), int64(7)
memory usage: 10.3 MB


In [216]:
#train.describe()
#test.describe()
#df.describe()
#train.to_csv("./data/credit-data-trainingset-hx.csv", index=False)
#test.to_csv("./data/credit-data-testset-hx.csv", index=False)



8       3950.450121
16      7531.592409
32      5741.021265
41      4605.249757
58      6579.198173
71      7138.712628
86      7269.672555
96      3950.450121
100     4212.369976
109    10196.015207
118     4212.369976
124     7400.632482
130     5479.101411
141     6395.820901
143     4212.369976
...
149923    8317.351972
149931    6264.860974
149932    8710.231753
149940    4474.289830
149946    3950.450121
149948    3819.490194
149952    6972.077954
149953    5348.141484
149958    6133.901047
149960    3950.450121
149963    6883.985863
149967    4343.329903
149984    4343.329903
149992    8242.405934
149997    7312.540391
Name: monthly_income, Length: 22148, dtype: float64

6     NaN
8     NaN
16    NaN
32    NaN
41    NaN
52    NaN
58    NaN
62    NaN
71    NaN
86    NaN
92    NaN
96    NaN
100   NaN
109   NaN
115   NaN
...
149946   NaN
149948   NaN
149952   NaN
149953   NaN
149958   NaN
149960   NaN
149963   NaN
149965   NaN
149967   NaN
149975   NaN
149976   NaN
149977   NaN
149984   NaN
149992   NaN
149997   NaN
Name: monthly_income, Length: 29731, dtype: float64

##Building a Model
Since we have a fairly large volume of data (150K), we're going to build a predictive model that will enable us to automatically score each applicant. We will provide each applicant with a credit score. This will give us an easy to interpret, human readable form of the model.

##Our Strategy
If we're building a model, we're going to need a way to know whether or not it's working. Convincing other is oftentimes the most challenging parts of an analysis. Making repeatable, well documented work with clear success metrics makes all the difference.

For our classifier, we're going to use the following build methodology:

In [6]:
from IPython.core.display import Image
Image(url="https://s3.amazonaws.com/demo-datasets/traintest.png")

##Other Things to consider

- [Precision and Recall](http://en.wikipedia.org/wiki/Precision_and_recall) - What good is our classifier if declines everyone?
- [Overfitting](http://en.wikipedia.org/wiki/Overfitting) - Is your model describing noise or signal?
- [Algorithms](http://en.wikipedia.org/wiki/Statistical_classification#Algorithms) - What type of classifiers might work in this scenario?

##We just did the following

- Determined we had a problem in which there was ample room for improvement
- Decided we could use a predictive model to decrease losses
- Proposed a high level workflow